In [8]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [9]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [10]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [11]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [12]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [13]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [14]:
## 
n='Input_C_058' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_C_058
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.002021824433555399
test_rmse: 0.0020584156109494356
test_r2: 0.8517259042434858
feature number: 3
train_rmse: 0.0019674152287821023
test_rmse: 0.0019785009538908944
test_r2: 0.8627251402118764
feature number: 4
train_rmse: 0.0018916203702216718
test_rmse: 0.001877184772430306
test_r2: 0.876599019678776
feature number: 5
train_rmse: 0.0018708971621959492
test_rmse: 0.0018652485245889056
test_r2: 0.8783291999726703
feature number: 6
train_rmse: 0.0018478926277017578
test_rmse: 0.0018688158175240693
test_r2: 0.877729645419899
feature number: 7
train_rmse: 0.0018458259704691848
test_rmse: 0.0018819373102604716
test_r2: 0.8759966030686014
feature number: 8
train_rmse: 0.0018247132733021526
test_rmse: 0.001859606364127516
test_r2: 0.8787881871543896
feature number: 9
train_rmse: 0.0018062772782699791
test_rmse: 0.0018484704121483684
test_r2: 0.8802204885524739
feature number: 10
train_rmse: 0.0017960261669006487
test_rmse:

train_rmse: 0.0017798414808652431
test_rmse: 0.0018810622720061478
test_r2: 0.8757880977157381
feature number: 16
train_rmse: 0.0017785322706399594
test_rmse: 0.0018908049681695537
test_r2: 0.8744073848239221
feature number: 17
train_rmse: 0.0017731513925938586
test_rmse: 0.001894553970505368
test_r2: 0.8740038708644411
feature number: 18
train_rmse: 0.001769350941813091
test_rmse: 0.0019025439138239074
test_r2: 0.8730822792718069
feature number: 19
train_rmse: 0.0017686496055511996
test_rmse: 0.001907334365727657
test_r2: 0.8724568406216872
feature number: 20
train_rmse: 0.001765969377219313
test_rmse: 0.001914131824468291
test_r2: 0.8715266661260073
Test_rmse_min = 1.fea_num: 9  2.rmse: 0.0018484704121483684
Test_r2_max = 1. fea_num: 9  2.r2: 0.8802204885524739
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.002021824433555399
test_rmse: 0.0020584156109494356
test_r2: 0.8517259042434858
feature number: 3
train_rmse: 0.0019674152287821023
test_rmse: 0.0019785009538908944
tes

train_rmse: 0.0018458259704691848
test_rmse: 0.0018819373102604716
test_r2: 0.8759966030686014
feature number: 8
train_rmse: 0.0018247132733021526
test_rmse: 0.001859606364127516
test_r2: 0.8787881871543896
feature number: 9
train_rmse: 0.0018062772782699791
test_rmse: 0.0018484704121483684
test_r2: 0.8802204885524739
feature number: 10
train_rmse: 0.0017960261669006487
test_rmse: 0.001850520929147744
test_r2: 0.8798701740827923
feature number: 11
train_rmse: 0.0017947610078673018
test_rmse: 0.0018583202971988893
test_r2: 0.8788241107549407
feature number: 12
train_rmse: 0.0017908633934019228
test_rmse: 0.0018634491922886215
test_r2: 0.8781265678197778
feature number: 13
train_rmse: 0.001788601542681866
test_rmse: 0.0018637435085615336
test_r2: 0.8780755945049179
feature number: 14
train_rmse: 0.0017832075653443171
test_rmse: 0.0018710791170575114
test_r2: 0.877118323929983
feature number: 15
train_rmse: 0.0017798414808652431
test_rmse: 0.0018810622720061478
test_r2: 0.8757880977157381

# 可以修改第一層的神經元數量

In [17]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(8, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [16]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量 神經元6

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_5/Input_C_058.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.004993362769832486
[0.002059231748994227, 0.006156502571508256, 0.0028283735093551842, 0.014798490843919354, 0.0027555354327281393, 0.003154930617026988, 0.004231783087977788, 0.004392094264910565, 0.00683926691645341, 0.0027174187054509534]


test_r2_ave: -0.20754499228595308
[0.8240059282080658, -0.40541921900212263, 0.7316023062668677, -5.368636701192265, 0.7248757345133303, 0.7039025515707564, 0.3701737559356255, 0.3858902270732515, -0.7485391949861773, 0.7066946887531371]


feature number: 6
test_rmse_ave: 0.004277458052422016
[0.003901189657195341, 0.004567710927785161, 0.004277094377478344, 0.003625130756314933, 0.0036512644632518128, 0.0020972695373898545, 0.011553002489240244, 0.003734408548914174, 0.0033008516092373087, 0.002066658157412986]


test_r2_ave: 0.12697358891767274
[0.3683424136328084, 0.22636701110381108, 0.3862338616296227, 0.6178279069192287, 0.516937330602476, 0.8691530700927543, -3.694222706063554, 0.5560374258472324, 0.59270